© Newbies (2023). Made for Data Slayer 1.0

# 0. Prerequisites

In [ ]:
# Import Libraries
import pandas as pd
import gdown
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Download the Data

# 1. Train data
url_train = "https://drive.google.com/uc?id=1xZKL6syTkXuGY-ZhhRRQZbxh3i25oQJl"
train_file = 'train_data.csv'
gdown.download(url_train, train_file, quiet=False)

# 2. Test data
url_test = "https://drive.google.com/uc?id=1GLr2eXoJhM_m_jKr-4VmpDIIoqvQFYsF"
test_file = 'test_data.csv'
gdown.download(url_test, test_file, quiet=False)

# 3. Sample submission
url_sample = "https://drive.google.com/uc?id=1HZtYLlh2St8uM7_KtY5omROA3Drk-2Xf"
sample = 'sample_submission.csv'
gdown.download(url_sample, sample, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1xZKL6syTkXuGY-ZhhRRQZbxh3i25oQJl
To: /content/train_data.csv
100%|██████████| 12.2M/12.2M [00:00<00:00, 144MB/s]
Downloading...
From: https://drive.google.com/uc?id=1GLr2eXoJhM_m_jKr-4VmpDIIoqvQFYsF
To: /content/test_data.csv
100%|██████████| 4.92M/4.92M [00:00<00:00, 216MB/s]
Downloading...
From: https://drive.google.com/uc?id=1HZtYLlh2St8uM7_KtY5omROA3Drk-2Xf
To: /content/sample_submission.csv
100%|██████████| 588k/588k [00:00<00:00, 125MB/s]


'sample_submission.csv'

In [ ]:
train = pd.read_csv('train_data.csv')
train.head(3)

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb,CO2 Emissions(g/km)
0,1,FOLD,PICKUP TRUCK - STANDARD,3.5,6.0,A6,X,8.64 liters per 100 km,6.14 L/100 km,37.59 mpg Imp.,306.0
1,2,CHEVO,PICKUP TRUCK - STANDARD,5.3,NaN,A6,E,27.27 liters per 100 km,30.76 liters per 100 km,28.84 L/100 km,283.0
2,3,BMV,SUBCOMPACT,4.4,NaN,M6,Z,0.17 L/10km,2.03 liters per 100 km,zero,329.0


In [ ]:
test = pd.read_csv('test_data.csv')
test.head(3)

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb
0,137142,CADILUXE,MID-SIZE,3.6,NaN,AS8,X,6.40 km/L,1.77 L/100 km,25.04 MPG (AS)
1,137143,FOLD,VAN - PASSENGER,3.7,6.0,AS6,X,21.11 L/100km,14.38 liters per 100 km,5.53 km per L
2,137144,MITSU,SUV - SMALL,6.0,NaN,AV6,X,9.60 MPG (AS),27.45 mpg Imp.,12.99 MPG (AS)


In [ ]:
sample = pd.read_csv('sample_submission.csv')
sample

,Id,CO2 Emissions(g/km)
0,137142,0
1,137143,0
2,137144,0
3,137145,0
4,137146,0
...,...,...
58771,195913,0
58772,195914,0
58773,195915,0
58774,195916,0


# 1. Exploratory Data Analysis

In [ ]:
df = test

Mari kita lihat dataset yang diberikan dengan mengambil 5 sample awal.

In [ ]:
df.head(10)

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb
0,137142,CADILUXE,MID-SIZE,3.6,NaN,AS8,X,6.40 km/L,1.77 L/100 km,25.04 MPG (AS)
1,137143,FOLD,VAN - PASSENGER,3.7,6.0,AS6,X,21.11 L/100km,14.38 liters per 100 km,5.53 km per L
2,137144,MITSU,SUV - SMALL,6.0,NaN,AV6,X,9.60 MPG (AS),27.45 mpg Imp.,12.99 MPG (AS)
3,137145,LECUS,MID-SIZE,4.6,NaN,AS8,Z,1.68 L/10km,6.29 km/L,16.42 L/100km
4,137146,BMV,SUV - STANDARD,4.4,8.0,AS8,Z,5.65 MPG (AS),14.75 mpg Imp.,31.52 L/100km
5,137147,CHEVO,SUBCOMPACT,3.6,6.0,M6,X,35.25 km per L,22.67 MPG (AS),45.35 mpg Imp.
6,137148,FOLD,VAN - PASSENGER,3.5,6.0,AS10,X,15.33 mpg Imp.,15.13 mpg Imp.,1.85 L/10km
7,137149,TOLVO,SUV - STANDARD,2.0,4.0,AS8,Z,3.52 km per L,5.63 km per L,4.23 km/L
8,137150,TOYOTI,PICKUP TRUCK - STANDARD,1.4,4.0,AS6,X,16.59 km per L,31.50 MPG (AS),42.32 mpg Imp.
9,137151,MATSUDA,SUV - SMALL,2.5,4.0,AS6,X,5.27 km per L,21.45 L/100 km,20.09 liters per 100 km


**Disclaimer:** terdapat perbedaan satuan yang digunakan. Perlu dilakukan standarisasi

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58776 entries, 0 to 58775
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Id                     58776 non-null  int64 
 1   Make                   58776 non-null  object
 2   Vehicle Class          58243 non-null  object
 3   Engine Size(L)         51537 non-null  object
 4   Cylinders              52822 non-null  object
 5   Transmission           57621 non-null  object
 6   Fuel Type              58006 non-null  object
 7   Fuel Consumption City  57729 non-null  object
 8   Fuel Consumption Hwy   57781 non-null  object
 9   Fuel Consumption Comb  57641 non-null  object
dtypes: int64(1), object(9)
memory usage: 4.5+ MB


Terdapat 10 kolom (tanpa Id) dengan **10 kolom sebagai features** dan **1 kolom sebagai target,** yakni "CO2 Emissions(g/km)". Dengan **137141 baris** (Non-Null). **Disclaimer**: beberapa kolom bertipe objek sehingga mungkin perlu diubah nantinya sesuai dengan model yang akan digunakan.

Variabel target merupakan suatu **variabel kontinu,** maka selanjutnya akan digunakan model regresi untuk memprediksi variabel tersebut.

In [ ]:
# # plt.boxplot(df['CO2 Emissions(g/km)'])

# plt.title('Box plot of CO2 Emissions')
# plt.ylabel('CO2 Emissions (g/km)')

# plt.show()

Berdasarkan Box Plot tersebut, terlihat bahwa target variable terpusat di antara 200 hingga 300.

In [ ]:
# Q1 = df['CO2 Emissions(g/km)'].quantile(0.25)
# Q3 = df['CO2 Emissions(g/km)'].quantile(0.75)
# IQR = Q3 - Q1

# # Determine the boundaries for outliers
# lower_bound = Q1 - 1.5 * IQR
# upper_bound = Q3 + 1.5 * IQR

# # Count the outliers
# outliers = df[(df['CO2 Emissions(g/km)'] < lower_bound) | (df['CO2 Emissions(g/km)'] > upper_bound)]
# number_of_outliers = outliers.shape[0]
# print(f"% of outliers: {number_of_outliers/df['Id'].count()*100} ({number_of_outliers}) ")

Dan terdapat 4198 data outliers yang ada atau 3.061%. Beberapa steps yang dapat dilakukan nantinya:


1.   Transformasi data
2.   Trimming
3.   Imputasi



In [ ]:
missing_values = df.isnull().sum()

# Display the missing values for each column
print("Missing values in each column:")
print(missing_values)

Missing values in each column:
Id                          0
Make                        0
Vehicle Class             533
Engine Size(L)           7239
Cylinders                5954
Transmission             1155
Fuel Type                 770
Fuel Consumption City    1047
Fuel Consumption Hwy      995
Fuel Consumption Comb    1135
dtype: int64


Selanjutnya, terlihat banyaknya missing value di tiap kolom. Banyaknya missing value tidak merata pada tiap kolom, di mana kolom "Engine Size(L)" memiliki missing value terbanyak, yakni 16.619 dengan 137.141 total data, melakukan drop NA mungkin menyebabkan model kurang akurat. Maka, selanjutnya akan dilakukan beberapa cara untuk men-*treat*-nya seperti imputasi

In [ ]:
missing_rows = df["Fuel Consumption City"].isnull() | df["Fuel Consumption Hwy"].isnull() | df["Fuel Consumption Comb"].isnull()
total_missing_rows = missing_rows.sum()

print("Number of rows with missing values or NaN across the specified columns:", total_missing_rows)

condition_rows = df["Fuel Consumption Comb"].isnull() & ~df["Fuel Consumption City"].isnull() & ~df["Fuel Consumption Hwy"].isnull()
total_condition_rows = condition_rows.sum()

print("Number of rows where Comb is NaN but City and Hwy are NOT NaN:", total_condition_rows)

Number of rows with missing values or NaN across the specified columns: 3108
Number of rows where Comb is NaN but City and Hwy are NOT NaN: 1084


Banyaknya kolom dengan kondisi "Fuel Consumption City" ATAU "Fuel Consumption Hwy" ATAU "Fuel Consumption Comb" adalah 7442. Namun, kolom dengan "Fuel Consumption Comb" saja yang NaN bernilai 2490. Berdasarkan Metadata, Fuel Consumption Comb adalah 55% Fuel Consumption City and 45% Fuel Consumption Hwy. Kolom "Fuel Consumption Comb" akan diimputasi dengan menggunakan formula tersebut.


Beberapa data kategorikal yang kami deteksi adalah:  "Make", "Vehicle Class" "Transmission", dan "Fuel Type". Mari kita lihat persebarannya.

In [ ]:
df.groupby('Make')['Make'].count()

Make
ASURA            859
BARUSU          1673
BMV             6316
CADILUXE        1948
CHEVO           7049
DOGE            3024
FIAR             831
FOLD            7661
FOLKSWA         2411
GONDA           2627
JIPU            3195
KIO             2851
LAMBOGI          481
LAND CRAWLER    1029
LECUS           2189
MATSUDA         2208
MITSU           1135
NIRRAN          3146
RYUNDAI         2506
TOLVO           1484
TOYOTI          4153
Name: Make, dtype: int64

In [ ]:
df.groupby('Vehicle Class')['Vehicle Class'].count()

Vehicle Class
-1                            163
COMPACT                      9434
FULL-SIZE                    4320
MID-SIZE                     8886
MINICOMPACT                   726
MINIVAN                       828
PICKUP TRUCK - SMALL         1424
PICKUP TRUCK - STANDARD      4044
SPECIAL PURPOSE VEHICLE       612
STATION WAGON - MID-SIZE      364
STATION WAGON - SMALL        2542
SUBCOMPACT                   3825
SUV - SMALL                 12069
SUV - STANDARD               5168
TWO-SEATER                   1712
VAN - CARGO                   133
VAN - PASSENGER               502
missing                       182
na                            161
not-available                 186
not-recorded                  421
unestablished                 191
unknown                       169
unspecified                   181
Name: Vehicle Class, dtype: int64

In [ ]:
df.groupby('Transmission')['Transmission'].count()

Transmission
-1                 361
A10                173
A4                 548
A5                 915
A6                5181
A7                  11
A8                2846
A9                1561
AM6               1057
AM7               1466
AM8                148
AM9                 36
AS10              1852
AS5                343
AS6              12440
AS7                315
AS8               8566
AS9                719
AV                3204
AV10               125
AV6               1261
AV7               1213
AV8                388
M5                2031
M6                7757
M7                 115
missing            341
na                 397
not-available      358
not-recorded       791
unestablished      370
unknown            368
unspecified        364
Name: Transmission, dtype: int64

Beberapa data perlu di-clean, yakni "-1", "missing", "na", etc.

In [ ]:
df.groupby('Fuel Type')['Fuel Type'].count()

Fuel Type
-1                 266
D                 1184
E                 2852
N                   14
X                35863
Z                15785
missing            248
na                 243
not-available      285
not-recorded       498
unestablished      272
unknown            256
unspecified        240
Name: Fuel Type, dtype: int64

Beberapa data perlu di-clean, yakni "-1", "missing", "na", etc.

# 2. Data Cleaning

## Duplicated Data

In [ ]:
# Check for duplicated rows and determine the count
duplicated_rows = df[df.duplicated()]
duplicated_count = duplicated_rows.shape[0]; print(duplicated_count)

# Remove duplicated rows
df.drop_duplicates(inplace=True)

0


## Missing Values and Outliers Treatment

### Transmissions

In [ ]:
# Values to be dropped
values_to_drop = ["-1", "missing", "na", "not-available", "not-recorded", "unestablished", "unknown", "unspecified"]

# Drop the specified values from the 'Transmission' column
df = df[~df['Transmission'].isin(values_to_drop)]

### Fuel Type

In [ ]:
# Values to be dropped
values_to_drop = ["-1", "missing", "na", "not-available", "not-recorded", "unestablished", "unknown", "unspecified"]

# Drop the specified values from the 'Fuel Type' column
df = df[~df['Fuel Type'].isin(values_to_drop)]

### Engine Size(L)

Pada tahap awal, akan dicari tahu terlebih dahulu distribusi frekuensi tiap data pada Engine Size(L), Vehicle Class, dan Make.

In [ ]:
df.groupby('Engine Size(L)')['Engine Size(L)'].count()

Engine Size(L)
-1                351
1.0               110
1.2               206
1.3                68
1.4              1546
1.5              1212
1.6              1849
1.8              1553
2.0              8213
2.2                 9
2.3               468
2.4              3088
2.5              3701
2.7               753
2.8               146
3.0              3542
3.2               373
3.3               776
3.4                48
3.5              4787
3.6              3391
3.7               607
3.8               243
4.0               409
4.2                13
4.3               215
4.4              1027
4.6               245
4.8                23
5.0              1162
5.2               222
5.3              1110
5.4                59
5.6               163
5.7               634
5.8                 7
6.0               116
6.2               957
6.4               356
6.5               137
6.6                36
6.8                 8
8.4                39
missing           343
na               

In [ ]:
df.groupby('Vehicle Class')['Vehicle Class'].count()

Vehicle Class
-1                            146
COMPACT                      8531
FULL-SIZE                    3934
MID-SIZE                     8100
MINICOMPACT                   653
MINIVAN                       752
PICKUP TRUCK - SMALL         1290
PICKUP TRUCK - STANDARD      3671
SPECIAL PURPOSE VEHICLE       570
STATION WAGON - MID-SIZE      340
STATION WAGON - SMALL        2288
SUBCOMPACT                   3447
SUV - SMALL                 10937
SUV - STANDARD               4644
TWO-SEATER                   1546
VAN - CARGO                   114
VAN - PASSENGER               463
missing                       164
na                            150
not-available                 169
not-recorded                  367
unestablished                 176
unknown                       156
unspecified                   170
Name: Vehicle Class, dtype: int64

In [ ]:
df.groupby('Make')['Make'].count()

Make
ASURA            791
BARUSU          1512
BMV             5707
CADILUXE        1760
CHEVO           6413
DOGE            2715
FIAR             746
FOLD            6949
FOLKSWA         2180
GONDA           2363
JIPU            2898
KIO             2568
LAMBOGI          445
LAND CRAWLER     936
LECUS           1973
MATSUDA         2019
MITSU           1036
NIRRAN          2860
RYUNDAI         2273
TOLVO           1348
TOYOTI          3761
Name: Make, dtype: int64


Terlihat bahwa missing values dan data-data seperti "missing", "na", "not-available", "not-recorded", "unestablished", "unknown", "unspecified" pada kolom Vehicle Class tidak terlalu banyak, begitu pula dengan nilai yang seharusnya tidak mungkin seperti -1 dan "-1". Dengan demikian, baris-baris yang memuat nilai tersebut akan dihapus.

In [ ]:
drop = ["-1", "missing", "na", "not-available", "not-recorded", "unestablished", "unknown", "unspecified", -1]

for i in drop:
  df = df[df["Vehicle Class"] != i]

df.groupby("Vehicle Class")["Vehicle Class"].count()

Vehicle Class
COMPACT                      8531
FULL-SIZE                    3934
MID-SIZE                     8100
MINICOMPACT                   653
MINIVAN                       752
PICKUP TRUCK - SMALL         1290
PICKUP TRUCK - STANDARD      3671
SPECIAL PURPOSE VEHICLE       570
STATION WAGON - MID-SIZE      340
STATION WAGON - SMALL        2288
SUBCOMPACT                   3447
SUV - SMALL                 10937
SUV - STANDARD               4644
TWO-SEATER                   1546
VAN - CARGO                   114
VAN - PASSENGER               463
Name: Vehicle Class, dtype: int64

In [ ]:
import numpy as np

# Handling missing values
replaced = [-1, '-1', 'missing', 'na', 'not-available', 'not-recorded', 'unestablished', 'unknown', 'unspecified']
df['Engine Size(L)'] = df['Engine Size(L)'].replace(replaced, np.nan)
df['Engine Size(L)'] = df['Engine Size(L)'].astype(float)  # Convert to float if not already

# Replace missing values with group mean using transform
df['Engine Size(L)'] = df['Engine Size(L)'].fillna(df.groupby(['Make', 'Vehicle Class'])['Engine Size(L)'].transform('mean'))

df.head(15)

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb
0,137142,CADILUXE,MID-SIZE,3.6,NaN,AS8,X,6.40 km/L,1.77 L/100 km,25.04 MPG (AS)
1,137143,FOLD,VAN - PASSENGER,3.7,6.0,AS6,X,21.11 L/100km,14.38 liters per 100 km,5.53 km per L
2,137144,MITSU,SUV - SMALL,6.0,NaN,AV6,X,9.60 MPG (AS),27.45 mpg Imp.,12.99 MPG (AS)
3,137145,LECUS,MID-SIZE,4.6,NaN,AS8,Z,1.68 L/10km,6.29 km/L,16.42 L/100km
4,137146,BMV,SUV - STANDARD,4.4,8.0,AS8,Z,5.65 MPG (AS),14.75 mpg Imp.,31.52 L/100km
5,137147,CHEVO,SUBCOMPACT,3.6,6.0,M6,X,35.25 km per L,22.67 MPG (AS),45.35 mpg Imp.
6,137148,FOLD,VAN - PASSENGER,3.5,6.0,AS10,X,15.33 mpg Imp.,15.13 mpg Imp.,1.85 L/10km
7,137149,TOLVO,SUV - STANDARD,2.0,4.0,AS8,Z,3.52 km per L,5.63 km per L,4.23 km/L
8,137150,TOYOTI,PICKUP TRUCK - STANDARD,1.4,4.0,AS6,X,16.59 km per L,31.50 MPG (AS),42.32 mpg Imp.
9,137151,MATSUDA,SUV - SMALL,2.5,4.0,AS6,X,5.27 km per L,21.45 L/100 km,20.09 liters per 100 km


### Cylinders

Untuk membersihkan data pada kolom Cylinders, tinjau terlebih dahulu data apa saja sekaligus berapa banyak frekuensi data tersebut muncul.

In [ ]:
df.groupby('Cylinders')['Cylinders'].count()

Cylinders
-1                 295
10.0               255
12.0               226
3.0                361
4.0              25318
5.0                148
6.0              11810
8.0               5803
missing            288
na                 265
not-available      276
not-recorded       563
unestablished      273
unknown            345
unspecified        285
Name: Cylinders, dtype: int64

Data-data seperti -1, 'missing', 'na', 'not-available', 'not-recorded', 'unestablished', 'unknown', 'unspecified' tentu harus dibersihkan. Dalam hal ini, data-data tersebut akan diinput dengan modus data Cylinders. Hal ini atas pertimbangan bahwa data Cylinders merupakan data yang diskrit.

In [ ]:
import numpy as np

# Handling missing values
replaced = [-1, '-1', 'missing', 'na', 'not-available', 'not-recorded', 'unestablished', 'unknown', 'unspecified']
df['Cylinders'] = df['Cylinders'].replace(replaced, np.nan)
df['Cylinders'] = df['Cylinders'].astype(float)  # Convert to float if not already

# Replace missing values with mode using transform
df['Cylinders'] = df['Cylinders'].fillna(df.groupby(['Make', 'Vehicle Class'])['Cylinders'].transform(lambda x: x.mode()[0]))

df.head(15)


,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb
0,137142,CADILUXE,MID-SIZE,3.6,6.0,AS8,X,6.400000,56.497175,10.645602
1,137143,FOLD,VAN - PASSENGER,3.7,6.0,AS6,X,4.737091,6.954103,5.530000
2,137144,MITSU,SUV - SMALL,6.0,4.0,AV6,X,4.081381,9.717471,5.522619
3,137145,LECUS,MID-SIZE,4.6,6.0,AS8,Z,5.952381,6.290000,6.090134
4,137146,BMV,SUV - STANDARD,4.4,8.0,AS8,Z,2.402063,5.221592,3.172589
5,137147,CHEVO,SUBCOMPACT,3.6,6.0,M6,X,35.250000,9.638011,16.054183
6,137148,FOLD,VAN - PASSENGER,3.5,6.0,AS10,X,5.426916,5.356114,5.405405
7,137149,TOLVO,SUV - STANDARD,2.0,4.0,AS8,Z,3.520000,5.630000,4.230000
8,137150,TOYOTI,PICKUP TRUCK - STANDARD,1.4,4.0,AS6,X,16.590000,13.392031,14.981544
9,137151,MATSUDA,SUV - SMALL,2.5,4.0,AS6,X,5.270000,4.662005,4.977601


## Fuel Consumption City

In [ ]:
df['Fuel Consumption City'].head(15)

0                   6.40 km/L
1               21.11 L/100km
2               9.60 MPG (AS)
3                 1.68 L/10km
4               5.65 MPG (AS)
5              35.25 km per L
6              15.33 mpg Imp.
7               3.52 km per L
8              16.59 km per L
9               5.27 km per L
10    19.36 liters per 100 km
11             14.35 L/100 km
14             19.92 mpg Imp.
15                0.22 L/10km
16    15.83 liters per 100 km
Name: Fuel Consumption City, dtype: object

Terlihat bahwa satuan dan penulisan yang digunakan tidak terstandar. Oleh karena itu, perlu dilakukan standariasi. Standarisasi yang akan digunakan adalah **L/km**. Data type kolom ini object, akan diganti menjadi float.

### Satuan Km

Pertama-tama akan dicari yang menggunakan satuan km terlebih dahulu.

In [ ]:
fuel_city_km = df[df['Fuel Consumption City'].str.contains('km', na=False, case=False)]
fuel_city_km['Fuel Consumption City'].head(15)

0                   6.40 km/L
1               21.11 L/100km
3                 1.68 L/10km
5              35.25 km per L
7               3.52 km per L
8              16.59 km per L
9               5.27 km per L
10    19.36 liters per 100 km
11             14.35 L/100 km
15                0.22 L/10km
16    15.83 liters per 100 km
17             20.54 km per L
18                1.39 L/10km
19             16.81 L/100 km
20               2.03 L/100km
Name: Fuel Consumption City, dtype: object

Beberapa pola yang ditemukan:

1.   liters per 100 km
2.   L/10km
3.   km per L
4.   L/100km
5.   km/L
6.   L/100 km



In [ ]:
# Create subsets based on specific text in the "Fuel Consumption City" column
subset_liters_100km = fuel_city_km[fuel_city_km['Fuel Consumption City'].str.contains('liters per 100 km', na=False, case=False)]
subset_L_10km = fuel_city_km[fuel_city_km['Fuel Consumption City'].str.contains('L/10km', na=False, case=False)]
subset_km_per_L = fuel_city_km[fuel_city_km['Fuel Consumption City'].str.contains('km per L', na=False, case=False)]
subset_L_100km = fuel_city_km[fuel_city_km['Fuel Consumption City'].str.contains('L/100km', na=False, case=False)]
subset_km_L = fuel_city_km[fuel_city_km['Fuel Consumption City'].str.contains('km/L', na=False, case=False)]
subset_L_100_km = fuel_city_km[fuel_city_km['Fuel Consumption City'].str.contains('L/100 km', na=False, case=False)]

# Check the length of each subset
subset_lengths = [len(subset_liters_100km), len(subset_L_10km), len(subset_km_per_L), len(subset_L_100km), len(subset_km_L), len(subset_L_100_km)]

# Confirm that the sum of the lengths is equal to the length of the original DataFrame
total_length = len(fuel_city_km)

if sum(subset_lengths) == total_length:
    print("The sum of the lengths of each subset equals the length of the original DataFrame.")
else:
    print("The sum of the lengths of each subset does not equal the length of the original DataFrame.")

The sum of the lengths of each subset equals the length of the original DataFrame.


Dapat dikonfirmasi bahwa memang hanya terdapat 6 pola di atas saja yang ada di dalam data. Selanjutnya, akan dilakukan standarisasi menjadi Km/L

In [ ]:
# 1. liters per 100 km -> km/L

# Extract the numeric values and convert to float
subset_liters_100km['Fuel Consumption City'] = pd.to_numeric(subset_liters_100km['Fuel Consumption City'].str.extract('(\d+\.\d+)', expand=False), errors='coerce')

# Convert 'L/100km' to 'km/L' and multiply by 100
subset_liters_100km['Fuel Consumption City'] = 1 / subset_liters_100km['Fuel Consumption City'] * 100

<ipython-input-299-de92d864386d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_liters_100km['Fuel Consumption City'] = pd.to_numeric(subset_liters_100km['Fuel Consumption City'].str.extract('(\d+\.\d+)', expand=False), errors='coerce')
<ipython-input-299-de92d864386d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_liters_100km['Fuel Consumption City'] = 1 / subset_liters_100km['Fuel Consumption City'] * 100


In [ ]:
# 2. L/10km -> km/L

# Extract the numeric values and convert to float
subset_L_10km['Fuel Consumption City'] = pd.to_numeric(subset_L_10km['Fuel Consumption City'].str.extract('(\d+\.\d+)', expand=False), errors='coerce')

# Convert 'L/10km' to 'km/L' and multiply by 10
subset_L_10km['Fuel Consumption City'] = 1 / subset_L_10km['Fuel Consumption City'] * 10

<ipython-input-300-c28f2d8e3bf7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_L_10km['Fuel Consumption City'] = pd.to_numeric(subset_L_10km['Fuel Consumption City'].str.extract('(\d+\.\d+)', expand=False), errors='coerce')
<ipython-input-300-c28f2d8e3bf7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_L_10km['Fuel Consumption City'] = 1 / subset_L_10km['Fuel Consumption City'] * 10


In [ ]:
# 3. km per L (no change)

# Extract the numeric values and convert to float
subset_km_per_L['Fuel Consumption City'] = pd.to_numeric(subset_km_per_L['Fuel Consumption City'].str.extract('(\d+\.\d+)', expand=False), errors='coerce')

<ipython-input-301-1759237a86e9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_km_per_L['Fuel Consumption City'] = pd.to_numeric(subset_km_per_L['Fuel Consumption City'].str.extract('(\d+\.\d+)', expand=False), errors='coerce')


In [ ]:
# 4. L/100km -> km/L

# Extract the numeric values and convert to float
subset_L_100km['Fuel Consumption City'] = pd.to_numeric(subset_L_100km['Fuel Consumption City'].str.extract('(\d+\.\d+)', expand=False), errors='coerce')

# Convert 'L/100km' to 'km/L' and multiply by 100
subset_L_100km['Fuel Consumption City'] = 1 / subset_L_100km['Fuel Consumption City'] * 100

<ipython-input-302-7e1f36f5b01d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_L_100km['Fuel Consumption City'] = pd.to_numeric(subset_L_100km['Fuel Consumption City'].str.extract('(\d+\.\d+)', expand=False), errors='coerce')
<ipython-input-302-7e1f36f5b01d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_L_100km['Fuel Consumption City'] = 1 / subset_L_100km['Fuel Consumption City'] * 100


In [ ]:
# 5. km/L (no change)

# Extract the numeric values and convert to float
subset_km_L['Fuel Consumption City'] = pd.to_numeric(subset_km_L['Fuel Consumption City'].str.extract('(\d+\.\d+)', expand=False), errors='coerce')

<ipython-input-303-808b9377c4b8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_km_L['Fuel Consumption City'] = pd.to_numeric(subset_km_L['Fuel Consumption City'].str.extract('(\d+\.\d+)', expand=False), errors='coerce')


In [ ]:
# 6. L/100 km -> km/L

# Extract the numeric values and convert to float
subset_L_100_km['Fuel Consumption City'] = pd.to_numeric(subset_L_100_km['Fuel Consumption City'].str.extract('(\d+\.\d+)', expand=False), errors='coerce')

# Convert 'L/100km' to 'km/L' and multiply by 100
subset_L_100_km['Fuel Consumption City'] = 1 / subset_L_100_km['Fuel Consumption City'] * 100

<ipython-input-304-dfd1ec730a9d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_L_100_km['Fuel Consumption City'] = pd.to_numeric(subset_L_100_km['Fuel Consumption City'].str.extract('(\d+\.\d+)', expand=False), errors='coerce')
<ipython-input-304-dfd1ec730a9d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_L_100_km['Fuel Consumption City'] = 1 / subset_L_100_km['Fuel Consumption City'] * 100


In [ ]:
# Update the "Fuel Consumption City" column based on the modified subsets

df.loc[df['Id'].isin(subset_liters_100km['Id']), "Fuel Consumption City"] = subset_liters_100km["Fuel Consumption City"]
df.loc[df['Id'].isin(subset_L_10km['Id']), "Fuel Consumption City"] = subset_L_10km["Fuel Consumption City"]
df.loc[df['Id'].isin(subset_km_per_L['Id']), "Fuel Consumption City"] = subset_km_per_L["Fuel Consumption City"]
df.loc[df['Id'].isin(subset_L_100km['Id']), "Fuel Consumption City"] = subset_L_100km["Fuel Consumption City"]
df.loc[df['Id'].isin(subset_km_L['Id']), "Fuel Consumption City"] = subset_km_L["Fuel Consumption City"]
df.loc[df['Id'].isin(subset_L_100_km['Id']), "Fuel Consumption City"] = subset_L_100_km["Fuel Consumption City"]

In [ ]:
df.head(5)

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb
0,137142,CADILUXE,MID-SIZE,3.6,NaN,AS8,X,6.4,1.77 L/100 km,25.04 MPG (AS)
1,137143,FOLD,VAN - PASSENGER,3.7,6.0,AS6,X,4.737091,14.38 liters per 100 km,5.53 km per L
2,137144,MITSU,SUV - SMALL,6.0,NaN,AV6,X,9.60 MPG (AS),27.45 mpg Imp.,12.99 MPG (AS)
3,137145,LECUS,MID-SIZE,4.6,NaN,AS8,Z,5.952381,6.29 km/L,16.42 L/100km
4,137146,BMV,SUV - STANDARD,4.4,8.0,AS8,Z,5.65 MPG (AS),14.75 mpg Imp.,31.52 L/100km


Kolom tersebut telah berhasil distandarisasi menjadi Km/L.

### Satuan miles

Sama seperti pada kasus satuan Km sebelumnya, pertama-tama akan dicari yang menggunakan satuan miles terlebih dahulu.

In [ ]:
fuel_city_miles = df[df['Fuel Consumption City'].str.contains('mpg', na=False, case=False)]
fuel_city_miles['Fuel Consumption City'].head(10)

2      9.60 MPG (AS)
4      5.65 MPG (AS)
6     15.33 mpg Imp.
14    19.92 mpg Imp.
27    22.40 mpg Imp.
28    20.68 mpg Imp.
36    42.62 mpg Imp.
37    19.16 MPG (AS)
51    30.06 MPG (AS)
57    33.76 MPG (AS)
Name: Fuel Consumption City, dtype: object

Beberapa pola yang ditemukan :
1.   MPG (AS)
2.   mpg Imp.



In [ ]:
# Create subsets based on specific text in the "Fuel Consumption City" column
subset_mpg_Imp = fuel_city_miles[fuel_city_miles['Fuel Consumption City'].str.contains('Imp.', na=False, case=False)]
subset_MPG_AS = fuel_city_miles[fuel_city_miles['Fuel Consumption City'].str.contains('(AS)', na=False, case=False)]

# Check the length of each subset
subset_lengths = [len(subset_MPG_AS), len(subset_mpg_Imp)]

# Confirm that the sum of the lengths is equal to the length of the original DataFrame
total_length = len(fuel_city_miles)

if sum(subset_lengths) == total_length:
    print("The sum of the lengths of each subset equals the length of the original DataFrame.")
else:
    print("The sum of the lengths of each subset does not equal the length of the original DataFrame.")

The sum of the lengths of each subset equals the length of the original DataFrame.


<ipython-input-308-335d0fbb6e4d>:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  subset_MPG_AS = fuel_city_miles[fuel_city_miles['Fuel Consumption City'].str.contains('(AS)', na=False, case=False)]


Dapat dikonfirmasi bahwa memang hanya terdapat 2 pola di atas saja yang ada di dalam data. Selanjutnya, akan dilakukan standarisasi menjadi L/Km

In [ ]:
# 1. MPG (AS) -> km/L

# Extract the numeric values and convert to float
subset_MPG_AS['Fuel Consumption City'] = pd.to_numeric(subset_MPG_AS['Fuel Consumption City'].str.extract('(\d+\.\d+)', expand=False), errors='coerce')

# Convert 'MPG (AS)' to 'km/L' and multiply by 0,42514385
subset_MPG_AS['Fuel Consumption City'] = subset_MPG_AS['Fuel Consumption City'] * 0.42514385

<ipython-input-309-186670df5bb8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_MPG_AS['Fuel Consumption City'] = pd.to_numeric(subset_MPG_AS['Fuel Consumption City'].str.extract('(\d+\.\d+)', expand=False), errors='coerce')
<ipython-input-309-186670df5bb8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_MPG_AS['Fuel Consumption City'] = subset_MPG_AS['Fuel Consumption City'] * 0.42514385


In [ ]:
# 2. mpg Imp. -> Km/L

# Extract the numeric values and convert to float
subset_mpg_Imp['Fuel Consumption City'] = pd.to_numeric(subset_mpg_Imp['Fuel Consumption City'].str.extract('(\d+\.\d+)', expand=False), errors='coerce')

# Convert 'MPG (AS)' to 'km/L' and multiply by 0.35400623
subset_mpg_Imp['Fuel Consumption City'] = subset_mpg_Imp['Fuel Consumption City'] * 0.35400623

<ipython-input-310-1182c8c1b4d8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_mpg_Imp['Fuel Consumption City'] = pd.to_numeric(subset_mpg_Imp['Fuel Consumption City'].str.extract('(\d+\.\d+)', expand=False), errors='coerce')
<ipython-input-310-1182c8c1b4d8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_mpg_Imp['Fuel Consumption City'] = subset_mpg_Imp['Fuel Consumption City'] * 0.35400623


In [ ]:
# Update the "Fuel Consumption City" column based on the modified subsets

df.loc[df['Id'].isin(subset_MPG_AS['Id']), "Fuel Consumption City"] = subset_MPG_AS["Fuel Consumption City"]
df.loc[df['Id'].isin(subset_mpg_Imp['Id']), "Fuel Consumption City"] = subset_mpg_Imp["Fuel Consumption City"]

In [ ]:
# Convert the "Fuel Consumption City" to float, handling non-numeric values

df["Fuel Consumption City"] = pd.to_numeric(df["Fuel Consumption City"], errors='coerce')

In [ ]:
import numpy as np

# Handling missing values
replaced = [-1, '-1', 'missing', 'na', 'not-available', 'not-recorded', 'unestablished', 'unknown', 'unspecified']
df['Fuel Consumption City'] = df['Fuel Consumption City'].replace(replaced, np.nan)
df['Fuel Consumption City'] = df['Fuel Consumption City'].astype(float)  # Convert to float if not already

# Replace missing values with group mean using transform
df['Fuel Consumption City'] = df['Fuel Consumption City'].fillna(df.groupby(['Make', 'Vehicle Class'])['Fuel Consumption City'].transform('mean'))

df.head(15)

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb
0,137142,CADILUXE,MID-SIZE,3.6,NaN,AS8,X,6.400000,1.77 L/100 km,25.04 MPG (AS)
1,137143,FOLD,VAN - PASSENGER,3.7,6.0,AS6,X,4.737091,14.38 liters per 100 km,5.53 km per L
2,137144,MITSU,SUV - SMALL,6.0,NaN,AV6,X,4.081381,27.45 mpg Imp.,12.99 MPG (AS)
3,137145,LECUS,MID-SIZE,4.6,NaN,AS8,Z,5.952381,6.29 km/L,16.42 L/100km
4,137146,BMV,SUV - STANDARD,4.4,8.0,AS8,Z,2.402063,14.75 mpg Imp.,31.52 L/100km
5,137147,CHEVO,SUBCOMPACT,3.6,6.0,M6,X,35.250000,22.67 MPG (AS),45.35 mpg Imp.
6,137148,FOLD,VAN - PASSENGER,3.5,6.0,AS10,X,5.426916,15.13 mpg Imp.,1.85 L/10km
7,137149,TOLVO,SUV - STANDARD,2.0,4.0,AS8,Z,3.520000,5.63 km per L,4.23 km/L
8,137150,TOYOTI,PICKUP TRUCK - STANDARD,1.4,4.0,AS6,X,16.590000,31.50 MPG (AS),42.32 mpg Imp.
9,137151,MATSUDA,SUV - SMALL,2.5,4.0,AS6,X,5.270000,21.45 L/100 km,20.09 liters per 100 km


## Fuel Consumption Hwy & Fuel Consumption Comb

Kolom ini memiliki masalah dan cara penanganan yang sama dengan kolom 'Fuel Consumption City'. Untuk mengefisiensikan program, akan di-define suatu fungsi untuk melakukan standariasasi.

In [ ]:
# Define a function to process the fuel consumption data

def process_fuel_data(df, column):

    # Create subsets based on specific text in the column
    subset_liters_100km = df[df[column].str.contains('liters per 100 km', na=False, case=False)]
    subset_L_10km = df[df[column].str.contains('L/10km', na=False, case=False)]
    subset_km_per_L = df[df[column].str.contains('km per L', na=False, case=False)]
    subset_L_100km = df[df[column].str.contains('L/100km', na=False, case=False)]
    subset_km_L = df[df[column].str.contains('km/L', na=False, case=False)]
    subset_L_100_km = df[df[column].str.contains('L/100 km', na=False, case=False)]
    subset_MPG_AS = df[df[column].str.contains('(AS)', na=False, case=False)]
    subset_mpg_Imp = df[df[column].str.contains('Imp.', na=False, case=False)]

    # Convert "liters per 100 km" to "km/L"
    subset_liters_100km[column] = pd.to_numeric(subset_liters_100km[column].str.extract('(\d+\.\d+)', expand=False), errors='coerce')
    subset_liters_100km[column] = 1 / subset_liters_100km[column] * 100

    # Convert "L/10km" to "km/L"
    subset_L_10km[column] = pd.to_numeric(subset_L_10km[column].str.extract('(\d+\.\d+)', expand=False), errors='coerce')
    subset_L_10km[column] = 1 / subset_L_10km[column] * 10

    # Convert "km per L" to "km/L" (no change)
    subset_km_per_L[column] = pd.to_numeric(subset_km_per_L[column].str.extract('(\d+\.\d+)', expand=False), errors='coerce')

    # Convert "L/100km" to "km/L"
    subset_L_100km[column] = pd.to_numeric(subset_L_100km[column].str.extract('(\d+\.\d+)', expand=False), errors='coerce')
    subset_L_100km[column] = 1 / subset_L_100km[column] * 100

    # Convert "km/L" to "km/L" (no change)
    subset_km_L[column] = pd.to_numeric(subset_km_L[column].str.extract('(\d+\.\d+)', expand=False), errors='coerce')

    # Convert "L/100 km" to "km/L"
    subset_L_100_km[column] = pd.to_numeric(subset_L_100_km[column].str.extract('(\d+\.\d+)', expand=False), errors='coerce')
    subset_L_100_km[column] = 1 / subset_L_100_km[column] * 100

    # Convert "mpg lmp."" to "km/L"
    subset_mpg_Imp[column] = pd.to_numeric(subset_mpg_Imp[column].str.extract('(\d+\.\d+)', expand=False), errors='coerce')
    subset_mpg_Imp[column] = subset_mpg_Imp[column] * 0.35400623

    # Convert "MPG (AS)" to "km/L"
    subset_MPG_AS[column] = pd.to_numeric(subset_MPG_AS[column].str.extract('(\d+\.\d+)', expand=False), errors='coerce')
    subset_MPG_AS[column] = subset_MPG_AS[column] * 0.42514385

    # Update the column based on the modified subsets
    df.loc[df['Id'].isin(subset_liters_100km['Id']), column] = subset_liters_100km[column]
    df.loc[df['Id'].isin(subset_L_10km['Id']), column] = subset_L_10km[column]
    df.loc[df['Id'].isin(subset_km_per_L['Id']), column] = subset_km_per_L[column]
    df.loc[df['Id'].isin(subset_L_100km['Id']), column] = subset_L_100km[column]
    df.loc[df['Id'].isin(subset_km_L['Id']), column] = subset_km_L[column]
    df.loc[df['Id'].isin(subset_L_100_km['Id']), column] = subset_L_100_km[column]
    df.loc[df['Id'].isin(subset_MPG_AS['Id']), column] = subset_MPG_AS[column]
    df.loc[df['Id'].isin(subset_mpg_Imp['Id']), column] = subset_mpg_Imp[column]

# Apply the function to the "Fuel Consumption Hwy" and "Fuel Consumption Comb" columns
process_fuel_data(df, "Fuel Consumption Hwy")
process_fuel_data(df, "Fuel Consumption Comb")

<ipython-input-314-d4c19798c562>:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  subset_MPG_AS = df[df[column].str.contains('(AS)', na=False, case=False)]
<ipython-input-314-d4c19798c562>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_liters_100km[column] = pd.to_numeric(subset_liters_100km[column].str.extract('(\d+\.\d+)', expand=False), errors='coerce')
<ipython-input-314-d4c19798c562>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

In [ ]:
# Convert the "Fuel Consumption Hwy" & "Fuel Consumption Comb" column to float, handling non-numeric values
df["Fuel Consumption Hwy"] = pd.to_numeric(df["Fuel Consumption Hwy"], errors='coerce')
df["Fuel Consumption Comb"] = pd.to_numeric(df["Fuel Consumption Comb"], errors='coerce')

In [ ]:
import numpy as np

# Handling missing values
replaced = [-1, '-1', 'missing', 'na', 'not-available', 'not-recorded', 'unestablished', 'unknown', 'unspecified']
df['Fuel Consumption Hwy'] = df['Fuel Consumption Hwy'].replace(replaced, np.nan)
df['Fuel Consumption Hwy'] = df['Fuel Consumption Hwy'].astype(float)  # Convert to float if not already

# Replace missing values with group mean using transform
df['Fuel Consumption Hwy'] = df['Fuel Consumption Hwy'].fillna(df.groupby(['Make', 'Vehicle Class'])['Fuel Consumption Hwy'].transform('mean'))

df.head(15)

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb
0,137142,CADILUXE,MID-SIZE,3.6,NaN,AS8,X,6.400000,56.497175,10.645602
1,137143,FOLD,VAN - PASSENGER,3.7,6.0,AS6,X,4.737091,6.954103,5.530000
2,137144,MITSU,SUV - SMALL,6.0,NaN,AV6,X,4.081381,9.717471,5.522619
3,137145,LECUS,MID-SIZE,4.6,NaN,AS8,Z,5.952381,6.290000,6.090134
4,137146,BMV,SUV - STANDARD,4.4,8.0,AS8,Z,2.402063,5.221592,3.172589
5,137147,CHEVO,SUBCOMPACT,3.6,6.0,M6,X,35.250000,9.638011,16.054183
6,137148,FOLD,VAN - PASSENGER,3.5,6.0,AS10,X,5.426916,5.356114,5.405405
7,137149,TOLVO,SUV - STANDARD,2.0,4.0,AS8,Z,3.520000,5.630000,4.230000
8,137150,TOYOTI,PICKUP TRUCK - STANDARD,1.4,4.0,AS6,X,16.590000,13.392031,14.981544
9,137151,MATSUDA,SUV - SMALL,2.5,4.0,AS6,X,5.270000,4.662005,4.977601


In [ ]:
df.head(10)

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb
0,137142,CADILUXE,MID-SIZE,3.6,NaN,AS8,X,6.400000,56.497175,10.645602
1,137143,FOLD,VAN - PASSENGER,3.7,6.0,AS6,X,4.737091,6.954103,5.530000
2,137144,MITSU,SUV - SMALL,6.0,NaN,AV6,X,4.081381,9.717471,5.522619
3,137145,LECUS,MID-SIZE,4.6,NaN,AS8,Z,5.952381,6.290000,6.090134
4,137146,BMV,SUV - STANDARD,4.4,8.0,AS8,Z,2.402063,5.221592,3.172589
5,137147,CHEVO,SUBCOMPACT,3.6,6.0,M6,X,35.250000,9.638011,16.054183
6,137148,FOLD,VAN - PASSENGER,3.5,6.0,AS10,X,5.426916,5.356114,5.405405
7,137149,TOLVO,SUV - STANDARD,2.0,4.0,AS8,Z,3.520000,5.630000,4.230000
8,137150,TOYOTI,PICKUP TRUCK - STANDARD,1.4,4.0,AS6,X,16.590000,13.392031,14.981544
9,137151,MATSUDA,SUV - SMALL,2.5,4.0,AS6,X,5.270000,4.662005,4.977601


In [ ]:
df.loc[df['Fuel Consumption Comb'].isnull() & ~df['Fuel Consumption City'].isnull() & ~df['Fuel Consumption Hwy'].isnull(), 'Fuel Consumption Comb'] = 0.55 * df['Fuel Consumption City'] + 0.45 * df['Fuel Consumption Hwy']

Imputasi Fuel Consumption Comb jika nilai Fuel Consumption City dan Fuel Consumption Hwy diketahui.

## Encoding

In [ ]:
df.head(3)

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb
0,137142,CADILUXE,MID-SIZE,3.6,6.0,AS8,X,6.400000,56.497175,10.645602
1,137143,FOLD,VAN - PASSENGER,3.7,6.0,AS6,X,4.737091,6.954103,5.530000
2,137144,MITSU,SUV - SMALL,6.0,4.0,AV6,X,4.081381,9.717471,5.522619


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51755 entries, 0 to 58775
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Id                     51755 non-null  int64  
 1   Make                   51755 non-null  object 
 2   Vehicle Class          51280 non-null  object 
 3   Engine Size(L)         51677 non-null  float64
 4   Cylinders              51693 non-null  float64
 5   Transmission           50679 non-null  object 
 6   Fuel Type              51049 non-null  object 
 7   Fuel Consumption City  51465 non-null  float64
 8   Fuel Consumption Hwy   51201 non-null  float64
 9   Fuel Consumption Comb  51737 non-null  float64
dtypes: float64(5), int64(1), object(4)
memory usage: 6.4+ MB


In [ ]:
label_encoder = LabelEncoder()

# List of columns to label encode
columns_to_encode = ["Transmission", "Fuel Type", "Vehicle Class", "Make"]

encoded_mapping = {}
for column in columns_to_encode:
    label_encoder.fit(df[column].astype(str))
    encoded_mapping[column] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Print the encoded mapping for each column
for column, mapping in encoded_mapping.items():
    print(f"{column}:\n{mapping}\n")

Transmission:
{'A10': 0, 'A4': 1, 'A5': 2, 'A6': 3, 'A7': 4, 'A8': 5, 'A9': 6, 'AM6': 7, 'AM7': 8, 'AM8': 9, 'AM9': 10, 'AS10': 11, 'AS5': 12, 'AS6': 13, 'AS7': 14, 'AS8': 15, 'AS9': 16, 'AV': 17, 'AV10': 18, 'AV6': 19, 'AV7': 20, 'AV8': 21, 'M5': 22, 'M6': 23, 'M7': 24, 'nan': 25}

Fuel Type:
{'D': 0, 'E': 1, 'N': 2, 'X': 3, 'Z': 4, 'nan': 5}

Vehicle Class:
{'COMPACT': 0, 'FULL-SIZE': 1, 'MID-SIZE': 2, 'MINICOMPACT': 3, 'MINIVAN': 4, 'PICKUP TRUCK - SMALL': 5, 'PICKUP TRUCK - STANDARD': 6, 'SPECIAL PURPOSE VEHICLE': 7, 'STATION WAGON - MID-SIZE': 8, 'STATION WAGON - SMALL': 9, 'SUBCOMPACT': 10, 'SUV - SMALL': 11, 'SUV - STANDARD': 12, 'TWO-SEATER': 13, 'VAN - CARGO': 14, 'VAN - PASSENGER': 15, 'nan': 16}

Make:
{'ASURA': 0, 'BARUSU': 1, 'BMV': 2, 'CADILUXE': 3, 'CHEVO': 4, 'DOGE': 5, 'FIAR': 6, 'FOLD': 7, 'FOLKSWA': 8, 'GONDA': 9, 'JIPU': 10, 'KIO': 11, 'LAMBOGI': 12, 'LAND CRAWLER': 13, 'LECUS': 14, 'MATSUDA': 15, 'MITSU': 16, 'NIRRAN': 17, 'RYUNDAI': 18, 'TOLVO': 19, 'TOYOTI': 20}


In [ ]:
# Apply label encoding to the specified columns
for column in columns_to_encode:
    df[column] = label_encoder.fit_transform(df[column].astype(str))

Terlihat bahwa beberapa kolom merupakan data kategorikal. Kolom tersebut yakni, "Make", "Vehicle Class", "Transmission", dan "Fuel Type". Kolom-kolom tersebut akan di-encode dengan label encoding.

# 3. Feature Engineering

# 4. Model Building

In [ ]:
df.head()

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb
0,137142,3,2,3.6,6.0,15,3,6.400000,56.497175,10.645602
1,137143,7,15,3.7,6.0,13,3,4.737091,6.954103,5.530000
2,137144,16,11,6.0,4.0,19,3,4.081381,9.717471,5.522619
3,137145,14,2,4.6,6.0,15,4,5.952381,6.290000,6.090134
4,137146,2,12,4.4,8.0,15,4,2.402063,5.221592,3.172589


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51755 entries, 0 to 58775
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Id                     51755 non-null  int64  
 1   Make                   51755 non-null  int64  
 2   Vehicle Class          51755 non-null  int64  
 3   Engine Size(L)         51677 non-null  float64
 4   Cylinders              51693 non-null  float64
 5   Transmission           51755 non-null  int64  
 6   Fuel Type              51755 non-null  int64  
 7   Fuel Consumption City  51465 non-null  float64
 8   Fuel Consumption Hwy   51201 non-null  float64
 9   Fuel Consumption Comb  51737 non-null  float64
dtypes: float64(5), int64(5)
memory usage: 6.4 MB


In [ ]:
# Importing necessary libraries
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split the data into features and target variable
X = df[['Make', 'Vehicle Class', 'Engine Size(L)', 'Cylinders', 'Transmission', 'Fuel Type',
        'Fuel Consumption City', 'Fuel Consumption Hwy', 'Fuel Consumption Comb']]
y = df['CO2 Emissions(g/km)']

# Perform one-hot encoding for categorical features (if needed)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate an XGBoost regressor
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

# Fit the model to the training data
xg_reg.fit(X_train, y_train)

# Predict on the test data
preds = xg_reg.predict(X_test)

# Evaluate the model's performance
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))


XGBoostError: ignored

Linear Regression


In [ ]:
from sklearn.linear_model import LinearRegression

df_temp = df;
df_temp.replace([np.inf, -np.inf], np.nan, inplace=True)
df_temp = df_temp.dropna()
df_temp = df_temp[~(df_temp < 0).any(axis=1)]
# Split the data into features and target variable
X = df_temp[['Make', 'Vehicle Class', 'Engine Size(L)', 'Cylinders', 'Transmission', 'Fuel Type',
        'Fuel Consumption City', 'Fuel Consumption Hwy', 'Fuel Consumption Comb']]
y = df_temp['CO2 Emissions(g/km)']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an instance of the linear regression model
linear_reg = LinearRegression()

# Fit the model on the training data
linear_reg.fit(X_train, y_train)

# Make predictions on the test data
y_pred = linear_reg.predict(X_test)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))


Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np

df_temp = df;
df_temp.replace([np.inf, -np.inf], np.nan, inplace=True)
df_temp = df_temp.dropna()
df_temp = df_temp[~(df_temp < 0).any(axis=1)]
# Split the data into features and target variable
X = df_temp[['Make', 'Vehicle Class', 'Engine Size(L)', 'Cylinders', 'Transmission', 'Fuel Type',
        'Fuel Consumption City', 'Fuel Consumption Hwy', 'Fuel Consumption Comb']]
y = df_temp['CO2 Emissions(g/km)']

# df_temp[df_temp.isna().any(axis=1)]
# np.isinf(df_temp).values.sum()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create gradient boost regressor object
gradientregressor = GradientBoostingRegressor(max_depth = 5, n_estimators = 50, learning_rate = 1.0)

# Train gradient boost regressor
modelgb = gradientregressor.fit(X_train,y_train)

# Predict the dataset
predgb = modelgb.predict(X_test)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, predgb))
print("RMSE: %f" % (rmse))

RMSE: 51.589085


In [ ]:
import pickle
with open('regression.pkl', 'wb') as file:
    pickle.dump(modelgb, file)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np


# Split the data into features and target variable
X = df[['Make', 'Vehicle Class', 'Engine Size(L)', 'Cylinders', 'Transmission', 'Fuel Type',
        'Fuel Consumption City', 'Fuel Consumption Hwy', 'Fuel Consumption Comb']]


# Predict the dataset
predgb = modelgb.predict(X_test)

In [ ]:
    train.to_csv('test.csv', index=False)


ini ngecek data doang gais

In [ ]:
df[df.isna().any(axis=1)]

In [ ]:
df['Fuel Consumption Comb'].isna().sum()

In [ ]:
np.sum((df < 0).values.ravel())

In [ ]:
df.groupby('Fuel Consumption Comb')['Fuel Consumption Comb'].count()

In [ ]:
from joblib import dump, load

dump(df, 'cleaned_data_train.joblib')
loaded_data_train = load('cleaned_data_train.joblib')